## Importar librerias necesarias


In [4]:
from pathlib import Path
import duckdb
import pandas as pd
import dash
from dash import dcc, html
from dash.dependencies import Input, Output
import plotly.express as px
import io

## Crear la base de datos y poblar las tablas con los dataset

In [5]:
# Define las rutas a los archivos CSV y el nombre de la base de datos DuckDB
csv_file1 = Path('./dataset/demanda-ltimos-aos.csv')
csv_file2 = Path('./dataset/generacin-ltimos-aos.csv')
db_file = 'energiadb.duckdb'

# Define los nombres que tendrán las tablas dentro de la base de datos
table_name1 = 'demanda'
table_name2 = 'generacion'

# cargar los datos de los dataset en las tablas
def cargar_csv_a_duckdb(db_path, files_to_import):
    """Crea la base de datos DuckDB y carga los CSV en tablas."""
    
    # Conectar a la base de datos (se crea si no existe)
    conn = duckdb.connect(database=db_path)
    print(f"✅ Conectado a la base de datos '{db_path}'.")

    for table_name, csv_file in files_to_import.items():
        if not Path(csv_file).exists():
            print(f"⚠️ ERROR: Archivo '{csv_file}' no encontrado. Saltando la carga de la tabla '{table_name}'.")
            continue
            
        
        conn.execute(f"""
            CREATE OR REPLACE TABLE {table_name} AS 
            SELECT * FROM read_csv_auto('{csv_file}')
        """)
        print(f"👍 Tabla '{table_name}' cargada desde '{csv_file}' en DuckDB.")

    return conn



def cargar_tablas_a_pandas(conn, table_name):
    """Consulta una tabla de DuckDB y la devuelve como un DataFrame."""
    print(f"Cargando tabla '{table_name}' a DataFrame de Pandas...")
    
    # Ejecuta una consulta SELECT * y usa fetchdf() para obtener el DataFrame
    query = f"SELECT * FROM {table_name}"
    df = conn.execute(query).fetchdf()
    
    print(f"✨ DataFrame '{table_name}_df' creado con {len(df)} filas.")
    return df

# Ejecucion del flujo

# Diccionario de archivos a importar: {nombre_de_la_tabla: nombre_del_csv}
files_config = {
    table_name1: csv_file1,
    table_name2: csv_file2
}

# 1. Cargar datos a la base de datos y obtener la conexión
conn = cargar_csv_a_duckdb(db_file, files_config)

print("\n--- INICIO DE RECUPERACIÓN DE DATAFRAMES ---")

# Cargar las tablas de DuckDB a DataFrames de Pandas
df_demanda = cargar_tablas_a_pandas(conn, table_name1)
df_generacion = cargar_tablas_a_pandas(conn, table_name2)

#Cerrar la conexión
conn.close()
print("\nConexión a DuckDB cerrada.")



✅ Conectado a la base de datos 'energiadb.duckdb'.
👍 Tabla 'demanda' cargada desde 'dataset\demanda-ltimos-aos.csv' en DuckDB.
👍 Tabla 'generacion' cargada desde 'dataset\generacin-ltimos-aos.csv' en DuckDB.

--- INICIO DE RECUPERACIÓN DE DATAFRAMES ---
Cargando tabla 'demanda' a DataFrame de Pandas...
✨ DataFrame 'demanda_df' creado con 40388 filas.
Cargando tabla 'generacion' a DataFrame de Pandas...
✨ DataFrame 'generacion_df' creado con 22444 filas.

Conexión a DuckDB cerrada.


## Verificacion de datos cargados en el dataframe de Demanda

In [6]:
# --- VERIFICACIÓN DE RESULTADOS dataframe demanda---
print("\n--- Verificación de DataFrame ---")
print(f"DataFrame Generacion (primeras 3 filas):\n{df_generacion.head(3)}")
print("\n----------------------------------")
print(f"DataFrame Generacion (información):\n")
df_generacion.info()


--- Verificación de DataFrame ---
DataFrame Generacion (primeras 3 filas):
       id  anio mes   maquina central    agente  \
0  481607  2017  01  3ARRDI01    3ARR  QUIL3A3A   
1  481608  2017  01  ABRODI01    ABRO  CTBROWNG   
2  481609  2017  01  ACAJTG01    CAPE  CAPEX-QA   

              agente_descripcion        region       pais  tipo_maquina  \
0  QUILMES - PLANTA TRES ARROYOS  BUENOS AIRES  Argentina  MOTOR DIESEL   
1           C.T. ALMIRANTE BROWN   GRAN BS.AS.  Argentina  MOTOR DIESEL   
2       CAPEX S.A. AUTOGENERADOR       COMAHUE  Argentina     TURBO GAS   

  fuente_generacion tecnologia categoria_hidraulica  \
0           Térmica         DI                 None   
1           Térmica         DI                 None   
2           Térmica         CC                 None   

               categoria_region  generacion_neta_MWh  \
0  Gba - Litoral - Buenos Aires                0.000   
1  Gba - Litoral - Buenos Aires              109.454   
2                       Comah

## Verificacion de datos cargados en el dataframe de Generacion

In [7]:
# --- VERIFICACIÓN DE RESULTADOS ---
print("\n--- Verificación de DataFrame ---")
print(f"DataFrame Demanda (primeras 3 filas):\n{df_demanda.head(3)}")
print("\n----------------------------------")
print(f"DataFrame Demanda (información):\n")
df_demanda.info()


--- Verificación de DataFrame ---
DataFrame Demanda (primeras 3 filas):
       id  anio mes agente_nemo           agente_descripcion tipo_agente  \
0  699232  2017  01    AARGTAOY  AEROP ARG 2000 - Aeroparque          GU   
1  699233  2017  01    ABRILHCY          ABRIL CLUB DE CAMPO          GU   
2  699234  2017  01    ACARQQ3Y     ASOC.COOP.ARG. - Quequén          GU   

         region     provincia    categoria_area categoria_demanda  \
0   GRAN BS.AS.  BUENOS AIRES  Gran Usuario MEM      Gran Usuario   
1   GRAN BS.AS.  BUENOS AIRES  Gran Usuario MEM      Gran Usuario   
2  BUENOS AIRES  BUENOS AIRES  Gran Usuario MEM      Gran Usuario   

                  tarifa             categoria_tarifa  demanda_MWh  \
0  GUMAS/AUTOGENERADORES  Industrial/Comercial Grande     1990.439   
1  GUMAS/AUTOGENERADORES  Industrial/Comercial Grande     1609.464   
2  GUMAS/AUTOGENERADORES  Industrial/Comercial Grande      421.334   

               fecha_proceso  lote_id_log indice_tiempo  
0 2020

## Agregacion de datos de Demanda y Generacion

In [8]:
df_anio_provincia_demanda = df_demanda.groupby(['anio','region','provincia']).agg(
    Suma_Total_demanda=('demanda_MWh', 'sum'),
    
).reset_index() 

df_anio_provincia_generacion = df_generacion.groupby(['anio','region']).agg(
    Suma_Total_generacion=('generacion_neta_MWh', 'sum'),
    
).reset_index() 
print(df_anio_provincia_demanda)
print(df_anio_provincia_generacion)

    anio        region       provincia  Suma_Total_demanda
0   2017  BUENOS AIRES    BUENOS AIRES        1.518567e+07
1   2017        CENTRO         CORDOBA        9.830799e+06
2   2017        CENTRO        SAN LUIS        1.645214e+06
3   2017       COMAHUE        LA PAMPA        9.104410e+05
4   2017       COMAHUE         NEUQUEN        2.322854e+06
..   ...           ...             ...                 ...
87  2020      NOROESTE           SALTA        3.491502e+05
88  2020      NOROESTE  SGO.DEL ESTERO        3.446556e+05
89  2020      NOROESTE         TUCUMAN        5.674354e+05
90  2020    PATAGONICA          CHUBUT        7.719910e+05
91  2020    PATAGONICA      SANTA CRUZ        1.797828e+05

[92 rows x 4 columns]
    anio         region  Suma_Total_generacion
0   2017   BUENOS AIRES           2.659328e+07
1   2017         CENTRO           5.229347e+06
2   2017        COMAHUE           1.775977e+07
3   2017           CUYO           6.724920e+06
4   2017  GENERAC MOVIL           

## Merge entre dataframe de generacion y demanda

In [9]:
df_Generacion_demanda = pd.merge(
    right=df_anio_provincia_generacion,
    left=df_anio_provincia_demanda,
    on=['anio','region'],
    how='left'
)

print(df_Generacion_demanda)#!/usr/bin/env python3

    anio        region       provincia  Suma_Total_demanda  \
0   2017  BUENOS AIRES    BUENOS AIRES        1.518567e+07   
1   2017        CENTRO         CORDOBA        9.830799e+06   
2   2017        CENTRO        SAN LUIS        1.645214e+06   
3   2017       COMAHUE        LA PAMPA        9.104410e+05   
4   2017       COMAHUE         NEUQUEN        2.322854e+06   
..   ...           ...             ...                 ...   
87  2020      NOROESTE           SALTA        3.491502e+05   
88  2020      NOROESTE  SGO.DEL ESTERO        3.446556e+05   
89  2020      NOROESTE         TUCUMAN        5.674354e+05   
90  2020    PATAGONICA          CHUBUT        7.719910e+05   
91  2020    PATAGONICA      SANTA CRUZ        1.797828e+05   

    Suma_Total_generacion  
0            2.659328e+07  
1            5.229347e+06  
2            5.229347e+06  
3            1.775977e+07  
4            1.775977e+07  
..                    ...  
87           1.829499e+06  
88           1.829499e+06  
89 

## Agregacion por año y provincia

In [10]:
df_anio_provincia_total = df_Generacion_demanda.groupby(['anio','provincia']).agg(
    Suma_Total_demanda=('Suma_Total_demanda', 'sum'),
    suma_Total_generacion=('Suma_Total_generacion', 'sum'),
    
).reset_index() 
print(df_anio_provincia_total)    

    anio       provincia  Suma_Total_demanda  suma_Total_generacion
0   2017    BUENOS AIRES        6.533676e+07           5.587869e+07
1   2017       CATAMARCA        2.059276e+06           1.369341e+07
2   2017           CHACO        2.812484e+06           2.002848e+07
3   2017          CHUBUT        4.238831e+06           4.569140e+06
4   2017         CORDOBA        9.830799e+06           5.229347e+06
..   ...             ...                 ...                    ...
83  2020        SAN LUIS        2.759745e+05           1.555255e+06
84  2020      SANTA CRUZ        1.797828e+05           1.398240e+06
85  2020        SANTA FE        2.143580e+06           2.301906e+06
86  2020  SGO.DEL ESTERO        3.446556e+05           1.829499e+06
87  2020         TUCUMAN        5.674354e+05           1.829499e+06

[88 rows x 4 columns]


## Agregacion por Región y Provincia

In [11]:
df_region_provincia = df_Generacion_demanda[['region', 'provincia']].drop_duplicates()
print(df_region_provincia)

          region       provincia
0   BUENOS AIRES    BUENOS AIRES
1         CENTRO         CORDOBA
2         CENTRO        SAN LUIS
3        COMAHUE        LA PAMPA
4        COMAHUE         NEUQUEN
5        COMAHUE       RIO NEGRO
6           CUYO         MENDOZA
7           CUYO        SAN JUAN
8    GRAN BS.AS.    BUENOS AIRES
9        LITORAL      ENTRE RIOS
10       LITORAL        SANTA FE
11       NORESTE           CHACO
12       NORESTE      CORRIENTES
13       NORESTE         FORMOSA
14       NORESTE        MISIONES
15      NOROESTE       CATAMARCA
16      NOROESTE           JUJUY
17      NOROESTE        LA RIOJA
18      NOROESTE           SALTA
19      NOROESTE  SGO.DEL ESTERO
20      NOROESTE         TUCUMAN
21    PATAGONICA          CHUBUT
22    PATAGONICA      SANTA CRUZ


## Agregación por Demanda y Tipo Categoria

In [12]:
df_anio_provincia_tipo_categoria = df_demanda.groupby(['anio','region','provincia','categoria_tarifa']).agg(
    Suma_Total_demanda=('demanda_MWh', 'sum'),
    
).reset_index() 
df_anio_provincia_tipo_categoria['anio']=df_anio_provincia_tipo_categoria['anio'].astype(str)
print(df_anio_provincia_tipo_categoria)

     anio        region     provincia             categoria_tarifa  \
0    2017  BUENOS AIRES  BUENOS AIRES                    Comercial   
1    2017  BUENOS AIRES  BUENOS AIRES  Industrial/Comercial Grande   
2    2017  BUENOS AIRES  BUENOS AIRES            Mercado a Término   
3    2017  BUENOS AIRES  BUENOS AIRES                  Residencial   
4    2017        CENTRO       CORDOBA                    Comercial   
..    ...           ...           ...                          ...   
351  2020    PATAGONICA        CHUBUT            Mercado a Término   
352  2020    PATAGONICA        CHUBUT                  Residencial   
353  2020    PATAGONICA    SANTA CRUZ                    Comercial   
354  2020    PATAGONICA    SANTA CRUZ  Industrial/Comercial Grande   
355  2020    PATAGONICA    SANTA CRUZ                  Residencial   

     Suma_Total_demanda  
0           4265102.533  
1           6131647.452  
2                -0.140  
3           4788915.518  
4           3564391.094  
.. 

## Agregación por Generación y Tipo Fuente

In [13]:
df_anio_region_generacion_fuente = df_generacion.groupby(['anio','region','fuente_generacion']).agg(
    Suma_Total_generacion=('generacion_neta_MWh', 'sum'),
    
).reset_index() 

df_anio_region_provincia_fuente = pd.merge(
    right=df_region_provincia,
    left=df_anio_region_generacion_fuente,
    on=['region'],
    how='left'
)

df_anio_region_provincia_fuente['anio']=df_anio_region_provincia_fuente['anio'].astype(str)
print(df_anio_region_provincia_fuente)

     anio        region fuente_generacion  Suma_Total_generacion     provincia
0    2017  BUENOS AIRES           Nuclear           5.716228e+06  BUENOS AIRES
1    2017  BUENOS AIRES         Renovable           7.829500e+01  BUENOS AIRES
2    2017  BUENOS AIRES           Térmica           2.087697e+07  BUENOS AIRES
3    2017        CENTRO        Hidráulica           5.090057e+05       CORDOBA
4    2017        CENTRO        Hidráulica           5.090057e+05      SAN LUIS
..    ...           ...               ...                    ...           ...
277  2020    PATAGONICA        Hidráulica           4.752341e+05    SANTA CRUZ
278  2020    PATAGONICA         Renovable           6.807154e+05        CHUBUT
279  2020    PATAGONICA         Renovable           6.807154e+05    SANTA CRUZ
280  2020    PATAGONICA           Térmica           2.422909e+05        CHUBUT
281  2020    PATAGONICA           Térmica           2.422909e+05    SANTA CRUZ

[282 rows x 5 columns]


## Construcción del Dashboard

In [14]:
df = df_anio_provincia_total

# Limpieza y estandarización de columnas
df.columns = ['anio', 'provincia', 'Suma_Total_demanda', 'Suma_Total_generacion']
#df = df.drop(columns=['ID']).reset_index(drop=True)

# Convertir 'anio' a string para que se maneje mejor en los filtros de Dash
df['anio'] = df['anio'].astype(str)

# Obtener opciones únicas para los filtros
available_years = sorted(df['anio'].unique())
available_provinces = sorted(df['provincia'].unique())

# --- Función para formatear KPIs 
def format_kpi(number):
    """Formatea un número con separador de miles '.' y sin decimales."""
    # Uso de f-string y truco de reemplazo para simular separador de miles español
    return f"{number:,.0f}".replace(",", "X").replace(".", ",").replace("X", ".")


# INICIALIZACIÓN DE LA APLICACIÓN DASH ---
app = dash.Dash(__name__, title="Dashboard Energía")

#  DEFINICIÓN DEL LAYOUT (ESTRUCTURA) DEL DASHBOARD ---

app.layout = html.Div(style={'backgroundColor': '#f8f9fa', 'padding': '20px'}, children=[
    
    html.H1(
        children='Dashboard de Energía de Argentina: Demanda vs. Generación',
        style={'textAlign': 'center', 'color': '#2c3e50', 'marginBottom': '30px'}
    ),
    
    # FILTROS
    html.Div([
        html.Div([
            html.Label('Selecciona el Año:', style={'fontWeight': 'bold'}),
            dcc.Dropdown(
                id='anio-filter',
                options=[{'label': i, 'value': i} for i in available_years],
                value=available_years[-1],
                clearable=False,
                style={'width': '95%'}
            ),
        ], style={'width': '48%', 'display': 'inline-block', 'marginRight': '4%'}),
        
        html.Div([
            html.Label('Selecciona la Provincia:', style={'fontWeight': 'bold'}),
            dcc.Dropdown(
                id='provincia-filter',
                options=[{'label': i, 'value': i} for i in available_provinces],
                value=available_provinces,
                multi=True,
                style={'width': '95%'}
            ),
        ], style={'width': '48%', 'display': 'inline-block'}),

    ], style={'padding': '10px', 'backgroundColor': '#ffffff', 'borderRadius': '8px', 'marginBottom': '20px'}),

    # -------------------------------------------------------------------
    # KPI SECTION 
    html.Div([
        
        # KPI 1: Demanda Total
        html.Div([
            html.H3("Demanda Total (MWh)", style={'textAlign': 'center', 'color': '#3498db', 'fontSize': '18px'}),
            html.Div(id='kpi-demanda-total', style={'fontSize': '36px', 'textAlign': 'center', 'fontWeight': 'bold', 'color': '#3498db'})
        ], style={
            'flex': '1', # Flexbox: toma una parte igual del espacio disponible
            'padding': '15px',
            'backgroundColor': '#e8f6f6',
            'borderRadius': '8px',
            'boxShadow': '0 4px 6px rgba(0, 0, 0, 0.1)',
            'marginRight': '20px' # Margen para separar visualmente
        }),

        # KPI 2: Generación Total
        html.Div([
            html.H3("Generación Total (MWh)", style={'textAlign': 'center', 'color': '#2ecc71', 'fontSize': '18px'}),
            html.Div(id='kpi-generacion-total', style={'fontSize': '36px', 'textAlign': 'center', 'fontWeight': 'bold', 'color': '#2ecc71'})
        ], style={
            'flex': '1', # Flexbox: toma una parte igual del espacio disponible
            'padding': '15px',
            'backgroundColor': '#e8f6f6',
            'borderRadius': '8px',
            'boxShadow': '0 4px 6px rgba(0, 0, 0, 0.1)',
        }),

    ], 
    # ESTILO KPI
    style={
        'display': 'flex',              # Activa Flexbox
        'justifyContent': 'space-between', # Distribuye el espacio entre los ítems
        'marginBottom': '30px', 
        'marginTop': '10px', 
    }),
    # -------------------------------------------------------------------

    # Contenedor del Gráfico
    html.Div([
        dcc.Graph(id='demanda-generacion-graph')
    ], style={'backgroundColor': '#ffffff', 'padding': '20px', 'borderRadius': '8px'}),
    
    ##### div de pie chart
    
    html.Div([

        html.Div([
            dcc.Graph(id='pie-chart-demanda')
        ], style={'width': '48%', 'display': 'inline-block', 'padding': '10px'}),

        html.Div([
            dcc.Graph(id='pie-chart-generacion')
        ], style={'width': '48%', 'display': 'inline-block', 'padding': '10px'}),

    ], style={'backgroundColor': '#ffffff','borderRadius': '8px','padding': '20px','marginTop': '30px'
}),


])

# CALLBACKS (INTERACTIVIDAD) ---

@app.callback(
    Output('demanda-generacion-graph', 'figure'),
    Output('kpi-demanda-total', 'children'),
    Output('kpi-generacion-total', 'children'),
    Output('pie-chart-demanda', 'figure'),
    Output('pie-chart-generacion', 'figure'),



    [
        Input('anio-filter', 'value'),
        Input('provincia-filter', 'value')
    ]
)
def update_graph_and_kpis(selected_year, selected_provinces):
    """
    Actualiza el gráfico y calcula los valores KPIs basados en los filtros.
    """
    
    if not selected_provinces:
        empty_fig = px.bar(title="Selecciona al menos una provincia")
        return empty_fig, "0", "0"
        
    filtered_df = df[
        (df['anio'] == selected_year) & 
        (df['provincia'].isin(selected_provinces))
    ]
    
    # CÁLCULO DE KPIS
    total_demanda = filtered_df['Suma_Total_demanda'].sum()
    total_generacion = filtered_df['Suma_Total_generacion'].sum()
    
    # Formatear los números
    demanda_kpi_text = format_kpi(total_demanda)
    generacion_kpi_text = format_kpi(total_generacion)

    # PREPARACIÓN DEL GRÁFICO
    df_melted = filtered_df.melt(
        id_vars=['provincia'], 
        value_vars=['Suma_Total_demanda', 'Suma_Total_generacion'],
        var_name='Tipo', 
        value_name='Valor'
    )
    
    fig = px.bar(
        df_melted, 
        x='provincia', 
        y='Valor', 
        color='Tipo',
        barmode='group',
        title=f'Demanda y Generación Energética por Provincia en el Año {selected_year}',
        labels={'provincia': 'Provincia', 'Valor': 'Valor Total (MWh)', 'Tipo': 'Tipo de Energía'},
        height=500,
        color_discrete_map={
             'Suma_Total_demanda': '#3498db',
             'Suma_Total_generacion': '#2ecc71'
        }
    )
    
    
    fig.update_layout(xaxis_tickangle=-45, plot_bgcolor='white')

   

    # PIE CHART DE DEMANDA

    df_dem_filtered = df_anio_provincia_tipo_categoria[
        (df_anio_provincia_tipo_categoria['anio'] == selected_year) &
        (df_anio_provincia_tipo_categoria['provincia'].isin(selected_provinces))
    ]
    df_dem_grouped = df_dem_filtered.groupby('categoria_tarifa')['Suma_Total_demanda'].sum().reset_index()
    pie_fig_dem = px.pie(
        df_dem_grouped,
        names='categoria_tarifa',
        values='Suma_Total_demanda',
        title=f'Distribución de Demanda por Categoria en {selected_year}',
        color_discrete_sequence=px.colors.sequential.Blues_r
    )

    ## pie chart de generacion

    df_pie_filtered = df_anio_region_provincia_fuente[
        (df_anio_region_provincia_fuente['anio'] == selected_year) &
        (df_anio_region_provincia_fuente['provincia'].isin(selected_provinces))
    ]


    

    df_pie_grouped = df_pie_filtered.groupby('fuente_generacion')['Suma_Total_generacion'].sum().reset_index()

    pie_fig = px.pie(
        df_pie_grouped,
        names='fuente_generacion',
        values='Suma_Total_generacion',
        title=f'Distribución de Generación por Fuente en {selected_year}',
        color_discrete_sequence=px.colors.sequential.Blugrn
    )

    
    
    # RETORNO DE TODOS LOS OUTPUTS
    return fig, demanda_kpi_text, generacion_kpi_text,pie_fig_dem, pie_fig

# --- 5. EJECUCIÓN DEL SERVIDOR DASH ---
if __name__ == '__main__':
    app.run(debug=True)

## Para ver en el navegador hacer click en el link de la celda:

http://127.0.0.1:8050/